<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

Host: deeplearning-vm
You are here: /home/sebelino7_gmail_com/pycharm_depl
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


In [2]:
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
print("Training results should now be reproducible.")


Training results should now be reproducible.


# Train model

In [15]:
import json
from training import TrainParams, NagParams
from datasets import DatasetParams
from search import run_search, SearchParams

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,
    trainval_size=400,
    validation_set_fraction=0.2,
)

training_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=11,
    optimizer=NagParams(
        learning_rate=None,  # Included in search
        weight_decay=None,  # Included in search
        momentum=None,  # Included in search
    ),
    freeze_layers=True,
    unfreezing_epochs=(None, None),  # Included in search
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
)

search_params = SearchParams(
    learning_rate=lambda trial: trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
    weight_decay=lambda trial: trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
    momentum=lambda trial: trial.suggest_float("momentum", 0.7, 0.99, log=False),
    unfreeze_epoch_1=lambda trial: trial.suggest_int("unfreeze_epoch_1", 1, 5),
    unfreeze_epoch_2=lambda trial, unfreeze_epoch_1: trial.suggest_int(
        "unfreeze_epoch_2",
        low=unfreeze_epoch_1 + 1,
        high=9
    ),
)

search_result = run_search(dataset_params, training_params, search_params, n_trials=10)

print("Search result:")
print(json.dumps(search_result, indent=2))



Running study with dataset parameters:
{
  "splitting_seed": 42,
  "shuffler_seed": 42,
  "batch_size": 32,
  "validation_set_fraction": 0.2,
  "trainval_size": 400,
  "binary": false
}
and training parameters:
{
  "seed": 42,
  "architecture": "resnet50",
  "optimizer": {
    "name": "nag"
  },
  "n_epochs": 11,
  "freeze_layers": true,
  "unfreezing_epochs": [],
  "validation_freq": 1
}
[INFO] A new study created in memory with name: no-name-d77e9b72-428f-4562-82e7-929d99f17d17
max_update_steps=110


Update step:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch [1/11], Loss: 3.5854, Train Acc: 2.81%, Val Acc: 7.50%
Epoch [2/11], Loss: 3.4440, Train Acc: 21.88%, Val Acc: 40.00%
Epoch [3/11], Loss: 3.3146, Train Acc: 44.69%, Val Acc: 53.75%
Epoch [4/11], Loss: 3.2127, Train Acc: 55.00%, Val Acc: 62.50%
Epoch [5/11], Loss: 3.0740, Train Acc: 66.25%, Val Acc: 72.50%
Epoch [6/11], Loss: 2.8611, Train Acc: 76.56%, Val Acc: 75.00%
Epoch [7/11], Loss: 2.6344, Train Acc: 81.56%, Val Acc: 78.75%
Epoch [8/11], Loss: 2.3753, Train Acc: 86.88%, Val Acc: 82.50%
Epoch [9/11], Loss: 2.1230, Train Acc: 89.38%, Val Acc: 86.25%
Epoch [10/11], Loss: 1.8715, Train Acc: 90.94%, Val Acc: 88.75%
Epoch [11/11], Loss: 1.6575, Train Acc: 91.56%, Val Acc: 90.00%
Total elapsed: 21.78s, average per update step: 0.13s
[INFO] Trial 0 finished with value: 0.9 and parameters: {'learning_rate': 0.00031489116479568613, 'weight_decay': 0.0007114476009343421, 'momentum': 0.9122782431253075, 'unfreeze_epoch_1': 3, 'unfreeze_epoch_2': 4}. Best is trial 0 with value: 0.9.
max_

Update step:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch [1/11], Loss: 3.6110, Train Acc: 3.75%, Val Acc: 2.50%
Epoch [2/11], Loss: 3.5897, Train Acc: 6.56%, Val Acc: 6.25%
Epoch [3/11], Loss: 3.5555, Train Acc: 8.44%, Val Acc: 12.50%
Epoch [4/11], Loss: 3.5257, Train Acc: 13.75%, Val Acc: 15.00%
Epoch [5/11], Loss: 3.5028, Train Acc: 17.19%, Val Acc: 17.50%
Epoch [6/11], Loss: 3.4654, Train Acc: 19.38%, Val Acc: 21.25%
Epoch [7/11], Loss: 3.4228, Train Acc: 28.12%, Val Acc: 27.50%
Epoch [8/11], Loss: 3.3901, Train Acc: 32.19%, Val Acc: 30.00%
Epoch [9/11], Loss: 3.3587, Train Acc: 35.94%, Val Acc: 33.75%
Epoch [10/11], Loss: 3.3306, Train Acc: 40.00%, Val Acc: 38.75%
Epoch [11/11], Loss: 3.2869, Train Acc: 43.12%, Val Acc: 43.75%
Total elapsed: 20.94s, average per update step: 0.13s
[INFO] Trial 1 finished with value: 0.4375 and parameters: {'learning_rate': 4.207053950287933e-05, 'weight_decay': 1.493656855461763e-06, 'momentum': 0.9511910822747311, 'unfreeze_epoch_1': 4, 'unfreeze_epoch_2': 8}. Best is trial 0 with value: 0.9.
max_u

...

Epoch [1/11], Loss: 2.5995, Train Acc: 58.75%, Val Acc: 95.00%
Epoch [2/11], Loss: 0.9132, Train Acc: 95.62%, Val Acc: 98.75%
Epoch [3/11], Loss: 0.5110, Train Acc: 95.31%, Val Acc: 98.75%
Epoch [4/11], Loss: 0.2920, Train Acc: 97.50%, Val Acc: 97.50%
Epoch [5/11], Loss: 0.1558, Train Acc: 98.12%, Val Acc: 98.75%
Epoch [6/11], Loss: 0.0953, Train Acc: 99.38%, Val Acc: 98.75%
Epoch [7/11], Loss: 0.0562, Train Acc: 99.69%, Val Acc: 97.50%
Epoch [8/11], Loss: 0.0390, Train Acc: 99.69%, Val Acc: 97.50%
Epoch [9/11], Loss: 0.0307, Train Acc: 99.69%, Val Acc: 100.00%
Epoch [10/11], Loss: 0.0249, Train Acc: 99.69%, Val Acc: 98.75%
Epoch [11/11], Loss: 0.0255, Train Acc: 99.69%, Val Acc: 98.75%
Total elapsed: 21.07s, average per update step: 0.15s
[INFO] Trial 7 finished with value: 0.9875 and parameters: {'learning_rate': 0.01712337597316398, 'weight_decay': 8.200518402245835e-06, 'momentum': 0.7283249130618513, 'unfreeze_epoch_1': 4, 'unfreeze_epoch_2': 7}. Best is trial 5 with value: 1.0.
m

Update step:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch [1/11], Loss: 3.6469, Train Acc: 3.75%, Val Acc: 1.25%
Epoch [2/11], Loss: 3.6432, Train Acc: 1.56%, Val Acc: 1.25%
🔪 Trial 8 pruned at epoch 2
Total elapsed: 3.69s, average per update step: 0.11s
[INFO] Trial 8 finished with value: 0.0125 and parameters: {'learning_rate': 3.077180271250682e-05, 'weight_decay': 3.058656666978529e-05, 'momentum': 0.7099726711234133, 'unfreeze_epoch_1': 5, 'unfreeze_epoch_2': 7}. Best is trial 5 with value: 1.0.
max_update_steps=110


Update step:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch [1/11], Loss: 3.1872, Train Acc: 43.75%, Val Acc: 87.50%
Epoch [2/11], Loss: 1.9883, Train Acc: 90.31%, Val Acc: 91.25%
Epoch [3/11], Loss: 1.1939, Train Acc: 92.81%, Val Acc: 96.25%
Epoch [4/11], Loss: 0.5576, Train Acc: 96.56%, Val Acc: 97.50%
Epoch [5/11], Loss: 0.3507, Train Acc: 96.56%, Val Acc: 97.50%
Epoch [6/11], Loss: 0.2316, Train Acc: 98.44%, Val Acc: 97.50%
Epoch [7/11], Loss: 0.1268, Train Acc: 99.69%, Val Acc: 97.50%
Epoch [8/11], Loss: 0.0898, Train Acc: 99.06%, Val Acc: 97.50%
Epoch [9/11], Loss: 0.0671, Train Acc: 99.69%, Val Acc: 97.50%
Epoch [10/11], Loss: 0.0543, Train Acc: 99.69%, Val Acc: 97.50%
Epoch [11/11], Loss: 0.0514, Train Acc: 99.69%, Val Acc: 98.75%
Total elapsed: 21.63s, average per update step: 0.13s
[INFO] Trial 9 finished with value: 0.9875 and parameters: {'learning_rate': 0.004467752817973905, 'weight_decay': 8.61257919259488e-06, 'momentum': 0.8508197261415651, 'unfreeze_epoch_1': 3, 'unfreeze_epoch_2': 5}. Best is trial 5 with value: 1.0.
Se